In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from scipy.linalg   import lstsq

In [2]:
# Псевдорешение (неоднородной) системы линейных уравнений
def solve_approximate(a,b,m,is_positive,round_to=2):
        # Найти псевдообратную матрицу a_pinv
        # Найти псевдорешение x=a_pinv*b
        if is_positive:
            x, err = nnls(a,b)
        else:
            x, err, rnk, s  = lstsq(a, b)
        # Итоговый рейтинг (чем больше значение, тем выше ранг, лучше объект)
        #погрешность решения системы
        #if isinstance(x[0],np.ndarray):
        #    x=np.squeeze(np.asarray(x)) #x=x.squeeze()
        #else:
        #    x=np.array(x)
        #y=np.round(a.dot(x)-b, 14)[0]
        # при совместной системе все y=0
        # сумма квадратов yi
        #err=0.0
        #for i in range(0,len(y)):
        #    err += y[i]*y[i]
        #err=np.sum(y ** 2)
        rez={}
        rez['x']=np.round(x[0:m], round_to)
        rez['err']=err
        return rez

In [3]:
# матрица парных сравнений из вектора (чем больше значение, тем лучше)
def getT(vect,IsSortRevers=False):
    if IsSortRevers:
        vect = -1*np.array(vect)
    m = len(vect)
    t=np.zeros((m, m))
    for i in range(0,m-1):
        for j in range(i+1,m):
            tt=0;
            if pd.isna(vect[i]):
                tt=0
            elif vect[i]>vect[j]:
                tt=1
            elif vect[i]<vect[j]:
                tt=-1
            t[i][j] = tt
            t[j][i] = -tt
    return t
# getT([ 15, 16,18,19],1)

In [4]:
# нормированная матрица N
def getNnor(Matrix):
    m = Matrix['m']
    Nnor=np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if Matrix['Np'][i][j]>0:
                Nnor[i][j] = 1.0*Matrix['Nm'][i][j]/Matrix['Np'][i][j] 
    return Nnor

In [5]:
# сумарные матрицы   df (столбец - вектор)
def getMatrix(df,IsSortRevers=False):
    n = len(df.columns)
    m = len(df[df.columns[0]])
    Matrix={}
    Matrix['n']=n
    Matrix['m']=m
    Matrix['N']=np.zeros((m, m)) #2.4
    Matrix['N0']=np.zeros((m, m))#2.5
    Matrix['Nm']=np.zeros((m, m))#2.6
    Matrix['Ns']=np.zeros((m, m))#2.7
    Matrix['Np']=np.zeros((m, m))#2.8
    for k in range(0,n):
        x = df[df.columns[k]]
        if IsSortRevers:
            x = -1*np.array(x)
        #print(x)
        #print(getT(x))
        for i in range(0,m-1): 
            for j in range(i+1,m):                   
                if pd.isna(x[i])==False and pd.isna(x[j])==False :
                    #print(i,j)
                    Matrix['Np'][i][j] += 1
                    Matrix['Np'][j][i] += 1
                    if x[i]>x[j]:
                        Matrix['N'][i][j] += 1
                    elif x[i]<x[j]:  
                        Matrix['N'][j][i] += 1
                    else:
                            Matrix['N0'][i][j] += 1
                            Matrix['N0'][j][i] += 1
                Matrix['Nm'][i][j] = Matrix['N'][i][j] - Matrix['N'][j][i]
                Matrix['Nm'][j][i] = -Matrix['Nm'][i][j]
                Matrix['Ns'][i][j] = Matrix['N'][i][j] + 0.5*Matrix['N0'][i][j]
                Matrix['Ns'][j][i] = Matrix['N'][j][i] + 0.5*Matrix['N0'][j][i]
    return Matrix

In [6]:
#Чеботарёв
def method_rows_sum_extension(Matrix): #Matrix 'Nm','Np'
    n=Matrix['n']
    m=Matrix['m']
    x_max = n*(m-1)
    e=1/(2*x_max)

    # строчные суммы
    b=np.zeros(m)
    for i in range(0,m): 
        for j in range(0,m): 
            b[i] +=  Matrix['Nm'][i][j]
    a=-1*Matrix['Np']+n
    for i in range(0,m): 
        a[i][i]=0
        for k in range(0,m): 
            if k!=i:
                a[i][i]+=-n+ Matrix['Np'][i][k]
    #ax=b
    a=e*a+np.matrix(np.eye(m, dtype=int))
    # сум x =0
    b=np.append(b, 0)
    a=np.append(a,np.ones((1, m)), axis = 0)
    # -x_max<=x<=x_max
    e0=np.zeros((m, m))
    e1=np.matrix(np.eye(m, dtype=int))
    e1_=-1*np.matrix(np.eye(m, dtype=int))

    b=np.append(b,-x_max*np.ones((1, m)))
    a=np.concatenate((a,np.zeros((len(a), m))),axis=1)
    ee=np.concatenate((e1,e1_),axis=1)
    a=np.append(a,ee, axis = 0)
    b=np.append(b,x_max*np.ones((1, m)))
    a=np.concatenate((a,np.zeros((len(a), m))),axis=1)
    ee=np.concatenate((e1,e0,e1),axis=1)
    a=np.append(a,ee, axis = 0)
    a=np.array(a)
    # Привели к виду ax=b  
    return solve_approximate(a,b,m,is_positive=False)

In [7]:
# Востановление пропусков в одном векторе
def method_rang_complit_one(x,x_,m,Nnor):
        Nprop = 0
        for i in range(0,m): 
            if pd.isna(x[i])==True:
                Nprop += 1
        if Nprop == 0:
            rez={}
            rez['x']=x
            rez['err']=0
            return rez
        b=np.zeros(m)   
        a=np.matrix(np.eye(m, dtype=int))
        for i in range(0,m): 
            if pd.isna(x[i])==False:
                b[i] = x[i]
            else:
                b[i] = x_[i]                  
        kProp = 0
        for i in range(0,m): 
            if pd.isna(x[i])==True:
                kProp += 1
                a0=np.zeros((m-1, m))
                a1=np.zeros((m-1, m-1))
                for j in range(0,m-1):
                    a0[j][i]=1
                    i1=0
                for j in range(0,m): 
                    if i!=j:
                        if pd.isna(x[j])==False:
                            b=np.append(b,x[j])
                        else:
                            b=np.append(b,x_[j])  
                        a1[i1][i1]=-Nnor[i][j]
                        i1 += 1

                e1 = np.zeros((m+(kProp-1)*(m-1), m-1))
                e1 = np.concatenate((a,e1),axis=1)
                e2 = np.zeros(( m-1,(kProp-1)*(m-1)))
                e2 = np.concatenate((a0,e2),axis=1)
                e2 = np.concatenate((e2,a1),axis=1)
                a=np.append(e1,e2, axis = 0)
                #print(a)
        # Привели к виду ax=b
        #print(a)
        #print(b)
        #print(m)
        return solve_approximate(a,b,m,is_positive=True)
    

In [8]:
# Востановление пропусков   (не возвращаю rez['err'])
def method_rang_complit(df,x_,Matrix):
    n=Matrix['n']
    m=Matrix['m']
    # веса условий
    #Nnor=getNnor(Matrix)
    #print(Nnor)
    Nnor=(1/n)*Matrix['Nm']
    #print(Nnor)
    #print(t_)
    #df2 = df.copy(deep=True)
    df2 = pd.DataFrame(columns=np.array(df.columns))
    data={}
    for k in range(0,n):
        x = df[df.columns[k]]
        #print(x)
        rez=method_rang_complit_one(x,x_,m,Nnor) 
        data[df.columns[k]] =rez['x']
        #df2 = df2.append({df.columns[k]: pd.Series(rez['x'])}, ignore_index=True)
    return pd.DataFrame(data, np.array(df.index))